In [3]:
import streamlit as st
import pandas as pd
import numpy as np

In [4]:
st.title('Mapping Education In Lebanon')

2025-09-14 04:42:51.026 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [5]:
#STEP1 - LOADING & CLEANING AND STANDARDIZING COLUMN NAMES

# Load datasets
df_resource = pd.read_csv('data/educationresource.csv')
df_level = pd.read_csv('data/educationlevel.csv')

# -----------------------------
# Step 1: Extract Governorate and District from refArea
# -----------------------------
def extract_location(url):
    if pd.isna(url):
        return pd.Series([None, None])
    name = url.split("/")[-1].replace("_", " ").strip()
    if "Governorate" in name:
        return pd.Series([name, None])
    elif "District" in name:
        return pd.Series([None, name])
    else:
        return pd.Series([None, None])

df_resource[['Governorate', 'District']] = df_resource['refArea'].apply(extract_location)
df_level[['Governorate', 'District']] = df_level['refArea'].apply(extract_location)

# -----------------------------
# Step 2: Clean Town names
# -----------------------------
def clean_town(town):
    if pd.isna(town):
        return town
    return town.strip().title()  # Strip spaces and normalize capitalization

df_resource['Town'] = df_resource['Town'].apply(clean_town)
df_level['Town'] = df_level['Town'].apply(clean_town)

# -----------------------------
# Step 3: Convert numeric columns to float/int
# -----------------------------
def convert_numeric(df):
    for col in df.columns:
        # Skip non-numeric columns
        if df[col].dtype == 'object' and col not in ['refArea', 'Observation URI', 'references', 'publisher', 'dataset', 'Town', 'Governorate', 'District']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

df_resource = convert_numeric(df_resource)
df_level = convert_numeric(df_level)

# -----------------------------
# Step 4: Simplify column names
# -----------------------------
df_resource.rename(columns={
    'Existence of educational resources - exists': 'EduResourceExists',
    'Existence of educational resources - does not exist': 'EduResourceMissing',
    'Type and size of educational resources - vocational institute': 'VocationalInstitutes',
    'Type and size of educational resources - public schools': 'PublicSchools',
    'Type and size of educational resources - universities': 'Universities',
    'Type and size of educational resources - private schools': 'PrivateSchools',
    'Nb of universities by type - Lebanese University branches': 'LebaneseUniBranches',
    'Nb of universities by type - Private universities': 'PrivateUniBranches',
    'Public school coverage index (number of schools per citizen)': 'PublicSchoolCoverage'
}, inplace=True)

df_level.rename(columns={
    'PercentageofEducationlevelofresidents-illeterate': 'Illiteracy',
    'PercentageofSchooldropout': 'SchoolDropout',
    'PercentageofEducationlevelofresidents-university': 'University',
    'PercentageofEducationlevelofresidents-secondary': 'Secondary',
    'PercentageofEducationlevelofresidents-intermediate': 'Intermediate',
    'PercentageofEducationlevelofresidents-vocational': 'Vocational',
    'PercentageofEducationlevelofresidents-elementary': 'Elementary',
    'PercentageofEducationlevelofresidents-highereducation': 'HigherEducation'
}, inplace=True)

# -----------------------------
# Step 5 (Optional): Fill missing Governorate using District mapping
# -----------------------------
district_to_governorate = {
    "Baabda District": "Mount Lebanon Governorate",
    "Aley District": "Mount Lebanon Governorate",
    "Metn District": "Mount Lebanon Governorate",
    "Keserwan District": "Mount Lebanon Governorate",
    "Jbeil District": "Mount Lebanon Governorate",
    "Chouf District": "Mount Lebanon Governorate",
    "Zgharta District": "North Governorate",
    "Koura District": "North Governorate",
    "Batroun District": "North Governorate",
    "Bcharre District": "North Governorate",
    "Miniyeh-Danniyeh District": "North Governorate",
    "Akkar District": "Akkar Governorate",
    "Saida District": "South Governorate",
    "Tyre District": "South Governorate",
    "Bint Jbeil District": "South Governorate",
    "Nabatieh District": "South Governorate",
    "Marjeyoun District": "South Governorate",
    "Hasbaya District": "South Governorate",
    "Beqaa District": "Beqaa Governorate",
    "West Beqaa District": "Beqaa Governorate",
    "Rashaya District": "Beqaa Governorate",
    "Zahle District": "Beqaa Governorate",
    "Baalbek District": "Baalbek-Hermel Governorate",
    "Hermel District": "Baalbek-Hermel Governorate"
}

for df in [df_resource, df_level]:
    df['Governorate'] = df.apply(
        lambda row: district_to_governorate.get(row['District'], row['Governorate']),
        axis=1
    )

# -----------------------------
print(df_resource.head())
print(df_level.head())


                                             refArea  EduResourceExists  \
0  https://dbpedia.org/page/Mount_Lebanon_Governo...                  0   
1         https://dbpedia.org/page/South_Governorate                  0   
2         https://dbpedia.org/page/Akkar_Governorate                  1   
3         https://dbpedia.org/page/North_Governorate                  0   
4        http://dbpedia.org/resource/Baabda_District                  1   

   VocationalInstitutes                                    Observation URI  \
0                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
1                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
2                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
3                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
4                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   

                                         references         publisher  \
0  http

In [6]:
# Saving cleaned datasets to see them locally and proof read if the cleaning worked
df_resource.to_csv('educationresource_cleaned.csv', index=False)
df_level.to_csv('educationlevel_cleaned.csv', index=False)

print("✅ Both cleaned datasets saved as 'educationresource_cleaned.csv' and 'educationlevel_cleaned.csv'.")


✅ Both cleaned datasets saved as 'educationresource_cleaned.csv' and 'educationlevel_cleaned.csv'.


In [7]:
# STEP 3 - NOW THAT EVERYTHING IS CLEANED AND STANDARDISED WE MERGE 

# Merge datasets on Governorate + District + Town
df_merged = pd.merge(
    df_resource,
    df_level,
    on=['Governorate', 'District', 'Town'],
    how='outer',  # use 'outer' to keep all rows from both datasets
    suffixes=('_resource', '_level')
)

# Optional: reorder columns for readability
cols_order = ['Governorate', 'District', 'Town', 'refArea_resource', 'refArea_level'] + \
             [col for col in df_merged.columns if col not in ['Governorate', 'District', 'Town', 'refArea_resource', 'refArea_level']]

df_merged = df_merged[cols_order]

# Save merged dataset
df_merged.to_csv('education_merged.csv', index=False)

print("✅ Datasets merged and saved as 'education_merged.csv'")
print(df_merged.head())


✅ Datasets merged and saved as 'education_merged.csv'
         Governorate District              Town  \
0  Akkar Governorate     None      Aain Ez-Zeit   
1  Akkar Governorate     None        Aain Tinta   
2  Akkar Governorate     None      Aain Yaaqoub   
3  Akkar Governorate     None            Aaiyat   
4  Akkar Governorate     None  Aakkar El-Aatiqa   

                             refArea_resource  \
0  https://dbpedia.org/page/Akkar_Governorate   
1  https://dbpedia.org/page/Akkar_Governorate   
2  https://dbpedia.org/page/Akkar_Governorate   
3  https://dbpedia.org/page/Akkar_Governorate   
4  https://dbpedia.org/page/Akkar_Governorate   

                                refArea_level  EduResourceExists  \
0  https://dbpedia.org/page/Akkar_Governorate                  1   
1  https://dbpedia.org/page/Akkar_Governorate                  1   
2  https://dbpedia.org/page/Akkar_Governorate                  1   
3  https://dbpedia.org/page/Akkar_Governorate                  1   
4  h

In [8]:

def audit_dataset(df, df_name="Dataset"):
    print(f"\n===== AUDIT REPORT: {df_name} =====\n")
    
    # 1️⃣ Missing values
    print("1️⃣ Missing Values per Column:")
    print(df.isna().sum())
    
    # 2️⃣ Duplicate rows
    print("\n2️⃣ Duplicate Rows:")
    print(f"Total duplicate rows: {df.duplicated().sum()}")
    
    # Duplicate towns within same district/governorate
    if all(col in df.columns for col in ['Governorate', 'District', 'Town']):
        dup_town = df.duplicated(subset=['Governorate', 'District', 'Town']).sum()
        print(f"Duplicate Town-District-Governorate entries: {dup_town}")
    
    # 3️⃣ Inspect categorical columns
    print("\n3️⃣ Unique Categorical Values (first 50 shown):")
    for col in df.select_dtypes(include='object').columns:
        unique_vals = df[col].unique()
        print(f"{col}: {unique_vals[:50]}")
    
    # 4️⃣ Numeric columns check
    print("\n4️⃣ Numeric Columns Summary:")
    numeric_cols = df.select_dtypes(include=np.number).columns
    print(df[numeric_cols].describe())
    
    # Negative values check for numeric columns
    print("\n5️⃣ Negative Values in Numeric Columns:")
    for col in numeric_cols:
        neg_count = (df[col] < 0).sum()
        if neg_count > 0:
            print(f"{col}: {neg_count} negative values")
    
    # 5️⃣ String formatting checks
    print("\n6️⃣ String Cleaning Suggestions:")
    for col in ['Town', 'District', 'Governorate']:
        if col in df.columns:
            weird_chars = df[col].str.contains(r'[^\x00-\x7F]', regex=True, na=False).sum()
            print(f"{col}: {weird_chars} entries with non-ASCII characters")
    
    print(f"\n===== END OF AUDIT: {df_name} =====\n")


# Example usage:
audit_dataset(df_resource, "Education Resources")
audit_dataset(df_level, "Education Levels")

# Optional: check mismatched towns between datasets
towns_resource = set(df_resource['Town'].dropna())
towns_level = set(df_level['Town'].dropna())

print("🔹 Towns in Resources but not in Levels:")
print(towns_resource - towns_level)

print("\n🔹 Towns in Levels but not in Resources:")
print(towns_level - towns_resource)



===== AUDIT REPORT: Education Resources =====

1️⃣ Missing Values per Column:
refArea                   0
EduResourceExists         0
VocationalInstitutes      0
Observation URI           0
references                0
publisher                 0
EduResourceMissing        0
LebaneseUniBranches       0
PublicSchoolCoverage      0
PublicSchools             0
Universities              0
Town                      0
PrivateUniBranches        0
dataset                   0
PrivateSchools            0
Governorate             317
District                448
dtype: int64

2️⃣ Duplicate Rows:
Total duplicate rows: 0
Duplicate Town-District-Governorate entries: 0

3️⃣ Unique Categorical Values (first 50 shown):
refArea: ['https://dbpedia.org/page/Mount_Lebanon_Governorate'
 'https://dbpedia.org/page/South_Governorate'
 'https://dbpedia.org/page/Akkar_Governorate'
 'https://dbpedia.org/page/North_Governorate'
 'http://dbpedia.org/resource/Baabda_District'
 'http://dbpedia.org/resource/Byblos_Distri

In [9]:
#There was some errors when i saw the merged csv with districts and governorates (encoding errors)
# Fix non-ASCII characters in Districts
def clean_district(district):
    if pd.isna(district):
        return district
    # Replace weird chars with spaces or standardize them
    return district.encode('latin1', errors='ignore').decode('utf-8', errors='ignore').replace('â\x80\x93','-').replace('Ã©','é').strip()

df_resource['District'] = df_resource['District'].apply(clean_district)
df_level['District'] = df_level['District'].apply(clean_district)


In [10]:
district_to_governorate.update({
    "Miniyeh–Danniyeh District": "North Governorate",
    "Zahlé District": "Beqaa Governorate",
    "Tripoli District, Lebanon": "North Governorate",
    "Western Beqaa District": "Beqaa Governorate"
})

for df in [df_resource, df_level]:
    df['Governorate'] = df.apply(
        lambda row: district_to_governorate.get(row['District'], row['Governorate']),
        axis=1
    )


In [11]:
print(df_resource['Governorate'].isna().sum(), df_level['Governorate'].isna().sum())
print(df_resource['District'].isna().sum(), df_level['District'].isna().sum())


210 210
448 448


In [12]:
#I fixed them and merged again
# # Merge on Governorate + District + Town
df_merged = pd.merge(
    df_resource,
    df_level,
    on=['Governorate', 'District', 'Town'],
    how='outer',  # keeps all rows from both datasets
    suffixes=('_resource', '_level')
)

# Optional: reorder columns for clarity
cols_order = ['Governorate', 'District', 'Town'] + \
             [col for col in df_merged.columns if col not in ['Governorate', 'District', 'Town']]

df_merged = df_merged[cols_order]

# Save merged dataset
df_merged.to_csv('education_merged_cleaned.csv', index=False)

print("✅ Datasets merged and saved as 'education_merged_cleaned.csv'")
print(df_merged.head())


✅ Datasets merged and saved as 'education_merged_cleaned.csv'
         Governorate District              Town  \
0  Akkar Governorate     None      Aain Ez-Zeit   
1  Akkar Governorate     None        Aain Tinta   
2  Akkar Governorate     None      Aain Yaaqoub   
3  Akkar Governorate     None            Aaiyat   
4  Akkar Governorate     None  Aakkar El-Aatiqa   

                             refArea_resource  EduResourceExists  \
0  https://dbpedia.org/page/Akkar_Governorate                  1   
1  https://dbpedia.org/page/Akkar_Governorate                  1   
2  https://dbpedia.org/page/Akkar_Governorate                  1   
3  https://dbpedia.org/page/Akkar_Governorate                  1   
4  https://dbpedia.org/page/Akkar_Governorate                  1   

   VocationalInstitutes                           Observation URI_resource  \
0                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
1                     1  http://linked.aub.edu.lb/CODEC/Lebanon/ob

In [13]:
# Fix encoding issues in District column
def fix_district_encoding(district):
    if pd.isna(district):
        return district
    # Replace common mis-encoded characters
    district = district.replace('√©','é').replace('â\x80\x93','-').strip()
    return district

for df in [df_resource, df_level]:
    df['District'] = df['District'].apply(fix_district_encoding)


In [14]:
# Update the mapping
district_to_governorate.update({
    "Bcharre District": "North Governorate",
    "Sidon District": "South Governorate",
    "Matn District": "Mount Lebanon Governorate",
    "Byblos District": "Mount Lebanon Governorate",
    "Zahlé District": "Beqaa Governorate"
})

# Refill Governorate column
for df in [df_resource, df_level]:
    df['Governorate'] = df.apply(
        lambda row: district_to_governorate.get(row['District'], row['Governorate']),
        axis=1
    )


In [15]:
print(df_resource[['District','Governorate']].drop_duplicates().sort_values('District'))
print(df_level[['District','Governorate']].drop_duplicates().sort_values('District'))


                      District                 Governorate
19               Aley District   Mount Lebanon Governorate
4              Baabda District   Mount Lebanon Governorate
10            Batroun District           North Governorate
40         Bint Jbeil District           South Governorate
8             Bsharri District                        None
5              Byblos District   Mount Lebanon Governorate
41            Hasbaya District           South Governorate
329            Hermel District  Baalbek-Hermel Governorate
12           Keserwan District   Mount Lebanon Governorate
13          Marjeyoun District           South Governorate
29               Matn District   Mount Lebanon Governorate
35   Miniyeh–Danniyeh District           North Governorate
9               Sidon District           South Governorate
195  Tripoli District, Lebanon           North Governorate
7                Tyre District           South Governorate
55      Western Beqaa District           Beqaa Governora

In [16]:
# Merge the cleaned datasets
df_merged = pd.merge(
    df_resource,
    df_level,
    on=['Governorate', 'District', 'Town'],
    how='outer',  # Keep all rows from both datasets
    suffixes=('_resource', '_level')
)

# Optional: reorder columns for readability
cols_order = ['Governorate', 'District', 'Town'] + \
             [col for col in df_merged.columns if col not in ['Governorate', 'District', 'Town']]
df_merged = df_merged[cols_order]

# Save the final merged dataset
df_merged.to_csv('education_merged_final.csv', index=False, encoding='utf-8')

# Quick check
print("✅ Final merged dataset saved as 'education_merged_final.csv'")
print(df_merged.head())


✅ Final merged dataset saved as 'education_merged_final.csv'
         Governorate District              Town  \
0  Akkar Governorate     None      Aain Ez-Zeit   
1  Akkar Governorate     None        Aain Tinta   
2  Akkar Governorate     None      Aain Yaaqoub   
3  Akkar Governorate     None            Aaiyat   
4  Akkar Governorate     None  Aakkar El-Aatiqa   

                             refArea_resource  EduResourceExists  \
0  https://dbpedia.org/page/Akkar_Governorate                  1   
1  https://dbpedia.org/page/Akkar_Governorate                  1   
2  https://dbpedia.org/page/Akkar_Governorate                  1   
3  https://dbpedia.org/page/Akkar_Governorate                  1   
4  https://dbpedia.org/page/Akkar_Governorate                  1   

   VocationalInstitutes                           Observation URI_resource  \
0                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
1                     1  http://linked.aub.edu.lb/CODEC/Lebanon/obs

In [17]:
# Fix encoding issues in District column
def fix_district_encoding(district):
    if pd.isna(district):
        return district
    # Replace known mis-encoded characters
    district = district.replace('√©','é').replace('â\x80\x93','-').strip()
    return district

for df in [df_resource, df_level]:
    df['District'] = df['District'].apply(fix_district_encoding)


In [18]:
# Extend district → governorate mapping
district_to_governorate.update({
    "Bsharri District": "North Governorate",
    "Zahlé District": "Beqaa Governorate"
})

# Refill Governorate column in both datasets
for df in [df_resource, df_level]:
    df['Governorate'] = df.apply(
        lambda row: district_to_governorate.get(row['District'], row['Governorate']),
        axis=1
    )


In [19]:
# Check the problematic districts
print(df_resource[df_resource['District'].isin(['Bsharri District','Zahlé District'])][['District','Governorate']])
print(df_level[df_level['District'].isin(['Bsharri District','Zahlé District'])][['District','Governorate']])


              District        Governorate
8     Bsharri District  North Governorate
44      Zahlé District  Beqaa Governorate
65      Zahlé District  Beqaa Governorate
77      Zahlé District  Beqaa Governorate
118     Zahlé District  Beqaa Governorate
164   Bsharri District  North Governorate
171     Zahlé District  Beqaa Governorate
182   Bsharri District  North Governorate
186   Bsharri District  North Governorate
218   Bsharri District  North Governorate
230   Bsharri District  North Governorate
253   Bsharri District  North Governorate
275     Zahlé District  Beqaa Governorate
285   Bsharri District  North Governorate
289   Bsharri District  North Governorate
294   Bsharri District  North Governorate
335     Zahlé District  Beqaa Governorate
359     Zahlé District  Beqaa Governorate
392     Zahlé District  Beqaa Governorate
415   Bsharri District  North Governorate
447     Zahlé District  Beqaa Governorate
451     Zahlé District  Beqaa Governorate
478   Bsharri District  North Gove

In [20]:
# Merge the cleaned datasets on Governorate + District + Town
df_merged = pd.merge(
    df_resource,
    df_level,
    on=['Governorate', 'District', 'Town'],
    how='outer',  # keeps all rows from both datasets
    suffixes=('_resource', '_level')
)

# Optional: reorder columns for readability
cols_order = ['Governorate', 'District', 'Town'] + \
             [col for col in df_merged.columns if col not in ['Governorate', 'District', 'Town']]

df_merged = df_merged[cols_order]

# Save the final merged dataset
df_merged.to_csv('education_merged_final.csv', index=False, encoding='utf-8')

print("✅ Final merged dataset saved as 'education_merged_final.csv'")
print(df_merged.head())


✅ Final merged dataset saved as 'education_merged_final.csv'
         Governorate District              Town  \
0  Akkar Governorate     None      Aain Ez-Zeit   
1  Akkar Governorate     None        Aain Tinta   
2  Akkar Governorate     None      Aain Yaaqoub   
3  Akkar Governorate     None            Aaiyat   
4  Akkar Governorate     None  Aakkar El-Aatiqa   

                             refArea_resource  EduResourceExists  \
0  https://dbpedia.org/page/Akkar_Governorate                  1   
1  https://dbpedia.org/page/Akkar_Governorate                  1   
2  https://dbpedia.org/page/Akkar_Governorate                  1   
3  https://dbpedia.org/page/Akkar_Governorate                  1   
4  https://dbpedia.org/page/Akkar_Governorate                  1   

   VocationalInstitutes                           Observation URI_resource  \
0                     0  http://linked.aub.edu.lb/CODEC/Lebanon/observa...   
1                     1  http://linked.aub.edu.lb/CODEC/Lebanon/obs

In [21]:
import pandas as pd
import plotly.graph_objects as go

# Load merged dataset
df = pd.read_csv('education_merged_final.csv', encoding='utf-8')

# Aggregate by district
df_plot = df.groupby('District').agg({
    'PublicSchools':'sum',
    'PrivateSchools':'sum',
    'Universities':'sum',
    'VocationalInstitutes':'sum',
    'Illiteracy':'mean',
    'Elementary':'mean',
    'Intermediate':'mean',
    'Secondary':'mean',
    'Vocational':'mean',
    'University':'mean',
    'HigherEducation':'mean'
}).reset_index()

# Calculate total resources for bars
df_plot['TotalResources'] = df_plot['PublicSchools'] + df_plot['PrivateSchools'] + df_plot['Universities'] + df_plot['VocationalInstitutes']

# Sort by total resources for better visualization
df_plot = df_plot.sort_values('TotalResources', ascending=False)

# Create figure
fig = go.Figure()

# Add bar for total resources
fig.add_trace(go.Bar(
    x=df_plot['District'],
    y=df_plot['TotalResources'],
    name='Total Resources',
    marker_color='lightskyblue',
    hovertemplate=
        "<b>%{x}</b><br>" +
        "Total Resources: %{y}<br>" +
        "Elementary: %{customdata[0]:.1f}%<br>" +
        "Intermediate: %{customdata[1]:.1f}%<br>" +
        "Secondary: %{customdata[2]:.1f}%<br>" +
        "Vocational: %{customdata[3]:.1f}%<br>" +
        "University: %{customdata[4]:.1f}%<br>" +
        "HigherEducation: %{customdata[5]:.1f}%",
    customdata=df_plot[['Elementary','Intermediate','Secondary','Vocational','University','HigherEducation']].values
))

# Add line for illiteracy
fig.add_trace(go.Scatter(
    x=df_plot['District'],
    y=df_plot['Illiteracy'],
    name='Illiteracy (%)',
    mode='lines+markers',
    marker_color='crimson',
    yaxis='y2'
))

# Update layout for dual axis
fig.update_layout(
    title='Educational Resources and Illiteracy by District',
    xaxis_tickangle=-45,
    yaxis=dict(title='Total Resources'),
    yaxis2=dict(
        title='Illiteracy (%)',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.01, y=0.99),
    hovermode='x'
)

fig.show()
